In [1]:
from collections import Counter
from math import log10
from numpy import zeros, array, dot
from numpy.linalg import norm

## БОЛЬШЕ НИКАКИХ ИМПОРТОВ ДЕЛАТЬ НЕЛЬЗЯ

## Зададим небольшой корпус текстов

In [2]:
corpus = ["о боже мама мама я схожу с ума", 
          "ее улыбка мама кругом голова",
          "о боже мама мама пьяный без вина", 
          "ее улыбка мама самая самая"]

corpus = list(map(lambda x: x.split(), corpus))

In [26]:
corpus

[['о', 'боже', 'мама', 'мама', 'я', 'схожу', 'с', 'ума'],
 ['ее', 'улыбка', 'мама', 'кругом', 'голова'],
 ['о', 'боже', 'мама', 'мама', 'пьяный', 'без', 'вина'],
 ['ее', 'улыбка', 'мама', 'самая', 'самая']]

## Проиндексируем корпус

In [6]:
def index_corpus(corpus):
    vocab_idx = {}
    idx = 0
    for text in corpus:
        for word in text:
            if word in vocab_idx:
                continue
            vocab_idx[word] = idx
            idx += 1
    return vocab_idx

In [7]:
vocab_idx  = index_corpus(corpus) # map a token into id
vocab_idx

{'о': 0,
 'боже': 1,
 'мама': 2,
 'я': 3,
 'схожу': 4,
 'с': 5,
 'ума': 6,
 'ее': 7,
 'улыбка': 8,
 'кругом': 9,
 'голова': 10,
 'пьяный': 11,
 'без': 12,
 'вина': 13,
 'самая': 14}

In [12]:
import numpy as np
a = np.array([1,2,3])
a[[1,2]] = 1
a

array([1, 1, 1])

## Binary vectorizer

In [15]:
def binary_vectorizer(tokens, vocab_idx):
    """tokens - list of words"""
    text_vector = zeros(len(vocab_idx))
    
    #SOME MAGIC HERE
    for word in tokens:
        text_vector[vocab_idx[word]] = 1
    
    return text_vector

In [16]:
text = "ее улыбка мама самая самая"
tokens = text.split()
print (binary_vectorizer(tokens, vocab_idx) )

[0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]


## Реализуйте СountVectorizer

In [17]:
def count_vectorizer(tokens, vocab_idx):
    """tokens - list of words"""
    text_vector = zeros(len(vocab_idx))
    
    # SOME MAGIC HERE
    for word in tokens:
        text_vector[vocab_idx[word]] += 1
    
    return text_vector

In [18]:
text = "ее улыбка мама самая самая"
tokens = text.split()
print (count_vectorizer(tokens, vocab_idx) )

[0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 2.]


# Term Frequency
TF  — это частотность термина, которая измеряет, насколько часто термин встречается в документе. В длинных документах термин может встретиться в больших количествах. Поэтому применяют относительные частоты — делят количество раз, когда нужный термин встретился в тексте, на общее количество слов в данном тексте.

In [23]:
def compute_tf(tokens):
    """tokens - list of words
    returns Counter with structure {word: tf}"""
    
    # SOME MAGIC HERE
    tf_text = Counter()
    
    for word in tokens:
        tf_text[word] += 1 
        
    return tf_text

In [24]:
text = "ее улыбка мама самая самая"
tokens = text.split()
print( compute_tf(tokens))

Counter({'самая': 2, 'ее': 1, 'улыбка': 1, 'мама': 1})


## Inverse Document Frequency
IDF — это обратная документная частота. Считается как логарифм от общего количества документов, делённого на количество документов, в которых встречается термин.

In [27]:
def compute_idf(tokens, corpus):
    """tokens - list of words
    corpus - list of lists with words as elements
    returns Counter with structure {word: idf}"""
    idf_text = Counter()
    
    corp_len = len(corpus)
    # SOME MAGIC HERE
    for word in tokens:
        cnt = 0
        for doc in corpus:
            if word in doc:
                cnt += 1
        idf_text[word] = log10(corp_len/cnt)
    return idf_text

In [28]:
text = "ее улыбка мама самая самая"
tokens = text.split()
print( compute_idf(tokens, corpus))

Counter({'самая': 0.6020599913279624, 'ее': 0.3010299956639812, 'улыбка': 0.3010299956639812, 'мама': 0.0})



## Скомбинируем и получим tf-idf

In [29]:
def compute_tfidf(corpus):
    """corpus - list of lists with words as elements
    returns list of dicts {word: tf-idf} for each document in corpus"""
    documents_list = []

    for text in corpus:
        computed_tf = compute_tf(text)
        computed_idf = compute_idf(text, corpus)
        tf_idf_dictionary = {}  # word: tf_idf_value
        
        # SOME MAGIC HERE
        for word in text:
            tf_idf_dictionary[word] = computed_tf[word] * computed_idf[word]    
            
        documents_list.append(tf_idf_dictionary)  # documents_list contains dict {word: tf_idf} for each sentence 

    return documents_list

In [30]:
for i in compute_tfidf(corpus):
    print(i)

{'о': 0.3010299956639812, 'боже': 0.3010299956639812, 'мама': 0.0, 'я': 0.6020599913279624, 'схожу': 0.6020599913279624, 'с': 0.6020599913279624, 'ума': 0.6020599913279624}
{'ее': 0.3010299956639812, 'улыбка': 0.3010299956639812, 'мама': 0.0, 'кругом': 0.6020599913279624, 'голова': 0.6020599913279624}
{'о': 0.3010299956639812, 'боже': 0.3010299956639812, 'мама': 0.0, 'пьяный': 0.6020599913279624, 'без': 0.6020599913279624, 'вина': 0.6020599913279624}
{'ее': 0.3010299956639812, 'улыбка': 0.3010299956639812, 'мама': 0.0, 'самая': 1.2041199826559248}


## Самостоятельно сделаем  tf-idf векторизатор

In [31]:
def tfidf_vectorizer(corpus, vocab_idx):
    """corpus - list of lists with words as elements
    vocab_idx - map {word: idx} with indexes in vector space
    returns list of lists with tf-idf vectors for each document"""
    tf_idf_corpus = compute_tfidf(corpus)
    corpus_vectorized = zeros((len(corpus), len(vocab_idx)))
    
    for i,sentence in enumerate(tf_idf_corpus):
        for word in sentence:
            
            #SOME MAGIC HERE
            corpus_vectorized[i, vocab_idx[word]] = sentence[word]

    return corpus_vectorized

In [32]:
vec = tfidf_vectorizer(corpus, vocab_idx)

In [36]:
vec[0]

array([0.30103   , 0.30103   , 0.        , 0.60205999, 0.60205999,
       0.60205999, 0.60205999, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

##### Какие 2 вектора из получившихся ближе друг к другу по косинусному расстоянию: vec\[0\] и  vec\[2\] или  vec\[1\] и  vec\[3\]? 

In [42]:
dist_02 = dot(vec[0], vec[2])/norm(vec[0])/norm([vec[2]])
print('DIST 02: ', dist_02)

dist_13 = dot(vec[1], vec[3])/norm(vec[1])/norm([vec[3]])
print('DIST 13: ', dist_13)

DIST 02:  0.12598815766974242
DIST 13:  0.14907119849998599


## Самостоятельно реализуем Hashing Vectorizer

In [43]:
K = 6 # Num of buckets for hashing
def string2bucket(s):
    return hash(s) % K

AttributeError: 'list' object has no attribute 'shape'

Примечание: достаточно провести хеширование токенов корпуса и результат подать на вход готовому tf-idf векторизатору

P.S. Не забудьте, что словарь слово-индекс изменится

In [ ]:
def hash_corpus(corpus):
    # ONE STRING OF CODE HERE
    hashed_corpus = zeros(corp)
    return hashed_corpus

def hashing_vectorizer(corpus):
    # 2-3 STRINGS OF CODE HERE
    return hashed_corpus_vectors

In [ ]:
print (hashing_vectorizer(corpus) )

In [ ]:
cos02 = dot(vec[0],vec[2])/(norm(vec[0])*norm(vec[2]))
cos13 = dot(vec[1],vec[3])/(norm(vec[1])*norm(vec[3]))

In [ ]:
cos02, cos13

##### Какие 2 вектора из получившихся ближе друг к другу по косинусному расстоянию: vec\[0\] и  vec\[2\] или  vec\[1\] и  vec\[3\]? 

Попробуйте спросить у коллег, какие значения cos02 и cos13 получились у них, или перезапустите ноутбук еще раз и посмотрите на результаты (если вы - интроверт). 

Почему результаты различаются?